In [1]:
import sys  
import pandas as pd
from datetime import datetime

In [63]:
mn_path = r'../data/variables/var-dia-TMN_CON-fill.zip'
mx_path = r'../data/variables/var-dia-TMX_CON-fill.zip'
cuenca_estacion_path = f'../data/stations/est_cuenca.csv'

md_out_path = r'../data/matrix/cuenca_T_MEDIA_CON_M.csv'

In [4]:
dtparser = lambda x: datetime.strptime(x, '%Y-%m-%d')
usecols = ['CodigoEstacion','Fecha','ValorFilled']
indexcols = ['Fecha']
df_mn = pd.read_csv(mn_path, parse_dates = ['Fecha'], date_parser = dtparser, usecols = usecols, index_col = indexcols)
df_mx = pd.read_csv(mx_path, parse_dates = ['Fecha'], date_parser = dtparser, usecols = usecols, index_col = indexcols)

In [5]:
df_mn = df_mn['2000':'2019']
df_mx = df_mx['2000':'2019']

In [6]:
df_mn = df_mn.reset_index().set_index(['CodigoEstacion', 'Fecha'])
df_mx = df_mx.reset_index().set_index(['CodigoEstacion', 'Fecha'])
df_mn_mx = pd.concat([df_mn, df_mx], axis = 1)
df_mn_mx.columns = ['TMN_CO', 'TMX_CO']

In [18]:
df_mn_mx['TMEDIA_CO'] = df_mn_mx.mean(axis = 1)

In [90]:
df_mn_mx = df_mn_mx.reset_index()
df_mn_mx = df_mn_mx.groupby(['CodigoEstacion', df_mn_mx.Fecha.dt.to_period('M')]).TMEDIA_CO.median().reset_index()
df_mn_mx = df_mn_mx.rename(columns = {'CodigoEstacion': 'codigo'})
df_mn_mx.head()

,codigo,Fecha,TMEDIA_CO
0,11025010,2000-01,16.30
1,11025010,2000-02,16.40
2,11025010,2000-03,16.70
3,11025010,2000-04,16.95
4,11025010,2000-05,16.80


In [92]:
df_est_cuenca = pd.read_csv(cuenca_estacion_path, dtype = {'CUENCA':'string', 'TMN_EST': 'string'}, usecols = ['CUENCA','TMN_EST'])
df_est_cuenca = df_est_cuenca.rename(columns = {'CUENCA':'cuenca', 'TMN_EST': 'codigo'})
df_est_cuenca = df_est_cuenca.dropna().groupby(['cuenca','codigo']).count().reset_index()
df_est_cuenca['codigo'] = df_est_cuenca['codigo'].astype(int)

df_mn_mx = df_mn_mx.merge(df_est_cuenca, on='codigo', how = 'inner' )
df_mn_mx = df_mn_mx.drop(df_mn_mx[df_mn_mx.cuenca.isna()].index)
df_mn_mx.head()

In [94]:
df_mn_mx.to_csv(md_out_path, index = False, date_format = '%Y-%m')

In [78]:
estacion = 52055030
df_mn_mx[(df_mn_mx.CodigoEstacion == estacion) 
         #& ((df_mn_mx.TMN_CO.notna() & df_mn_mx.TMX_CO.isna())
         #   | (df_mn_mx.TMN_CO.isna() & df_mn_mx.TMX_CO.notna()))
        ]
#dfe = df_mn_mx[(df_mn_mx.CodigoEstacion == estacion)]
#dfe.isna().groupby([dfe.Fecha.dt.year, dfe.Fecha.dt.month]).TMN_CO.sum().unstack(level=1)

,CodigoEstacion,Fecha,TMN_CO,TMX_CO,TMEDIA_CO
2841645,52055030,2000-01-01,15.8,26.0,20.90
2841646,52055030,2000-01-02,16.6,25.4,21.00
2841647,52055030,2000-01-03,14.8,25.5,20.15
2841648,52055030,2000-01-04,14.2,24.8,19.50
2841649,52055030,2000-01-05,15.2,24.7,19.95
...,...,...,...,...,...
2848945,52055030,2019-12-27,16.7,27.0,21.85
2848946,52055030,2019-12-28,17.8,25.4,21.60
2848947,52055030,2019-12-29,15.7,26.8,21.25
2848948,52055030,2019-12-30,15.7,25.4,20.55
